# Assignment 1 - Yelp data

First, import relevent libraries. Add useful functions from the "useful gems" lab.I have also included a helper function to tell me that my browser is not crashing.

In [2]:
import tensorflow as tf
import sys
import sklearn as sk
import pandas as pd
import json
import csv
import time, sys
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from scipy.stats import zscore
import sklearn.feature_extraction.text as sk_text
from sklearn.neighbors import KNeighborsRegressor
from collections.abc import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn import metrics
import winsound



# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)
    
#Show progress bar in loop
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)
    
# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

#Let me know when you're done!
def beep():
    duration = 500  # milliseconds
    freq = 1040  # Hz
    winsound.Beep(freq, duration)

loadFresh=0

First determine businesses have enough reviews to be relevent for training


In [3]:
if(loadFresh):
    #Convert raw business data into tsv
    outfile = open("businesses.tsv", 'w')
    sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
    sfile.writerow(['business_id','stars', 'review_count'])
    with open('yelp_dataset/business.json', encoding="utf-8") as f:
     i=0
     for line in f:
        row = json.loads(line)
        if(row['review_count']>=20):
            sfile.writerow([row['business_id'], row['stars'], (row['review_count'])])
        i=i+1
        update_progress(i / 192610)

    update_progress(1);
    outfile.close()

In [3]:
#load business table
business_df= pd.read_csv('businesses.tsv', delimiter ="\t", encoding="utf-8")
business_df=business_df.sort_values('business_id');
business_df

,business_id,stars,review_count
3041,--I7YYLada0tSLkORTHb5Q,3.5,96
4225,-092wE7j5HZOogMLAh40zA,3.0,92
4572,-1VaIJza42Hjev6ukacCNg,4.0,210
883,-1xuC540Nycht_iWFeJ-dw,4.5,465
1080,-4-MzST67P_jnX4mh3MIcw,4.0,20
...,...,...,...
4297,zyPGYeXF4XKCqNN1pjFWhg,4.5,163
1076,zyr5pzOs3SJIX3K-nvC2zg,4.5,74
3274,zz3CqZhNx2rQ_Yp6zHze-A,3.0,47
2265,zzOo9n22fBbKAhbSpMzggA,3.0,25


Found 57644 with enough reviews to be useful. All stored in business_df, ordered by business ID

## Review data
Open the review data and convert into a TSV with only the relevent information. Runs a progress bar while loading. To save space, only writes to file if the business has 20 reviews or more.

In [4]:
if(loadFresh):
    #Convert raw review data into TSV
    outfile = open("review_stars.tsv", 'w')
    sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
    sfile.writerow(['business_id','stars', 'text'])
    with open('yelp_dataset/review.json', encoding="utf-8") as f:
     i=0
     for line in f:
        row = json.loads(line)
        # Check if businessID if present in business table
        if(row['business_id'] in business_df['business_id'].tolist()):
            # some special char must be encoded in 'utf-8'
            sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])
        update_progress(i / 2000000)
        i=i+1

    update_progress(1);
    outfile.close()

In [5]:
#load review data
df= pd.read_csv('review_stars.tsv', delimiter ="\t", encoding="utf-8")
df

,business_id,stars,text
0,ujmEBvifdJM6h6RLv4wQIg,1.0,b'Total bill for this horrible service? Over $...
1,NZnhc2sEQy3RmzKTZnqtwQ,5.0,"b""I *adore* Travis at the Hard Rock's new Kell..."
2,WTqjgwHlXbSFevF32_DJVw,5.0,"b""I have to say that this office really has it..."
3,b1b1eb3uo-w561D0ZfCEiQ,1.0,b'Today was my second out of three sessions I ...
4,eU_713ec6fTGNO4BegRaww,4.0,b'I\'ll be the first to admit that I was not e...
...,...,...,...
2232051,BLvcA14BVJYSPjaVnNUKFg,5.0,b'Delicious locally roasted organic coffee. F...
2232052,gCElF94aCMjB2SeFOEHLXg,5.0,"b""I recently visited this location with my fam..."
2232053,oIFMGuV8C8yeaDrtwjc8Jw,1.0,b'This place is a rip off! Charge you 44 if yo...
2232054,9M4bfZJlTSXfzGoKnQzdww,5.0,b'We recently needed to have our patio roof re...


## Language preprocessing
First, aggregates all reviews under their business ID. Then we can run TFIDF on the sum of all the reviews in preparation to do language analysis.

In [6]:
#groups reviews by businessID
df_review_group=df.groupby('business_id')['text'].sum()
df_reviews = pd.DataFrame({'business_id':df_review_group.index, 'all_reviews':df_review_group.values})
df_reviews

#language preprocessing
vectorizer = sk_text.TfidfVectorizer(stop_words='english',
                             min_df=2, 
                             max_df=500)

#df_reviews['all_reviews'] = vectorizer.fit_transform(df_reviews['all_reviews'])
vector_reviews=vectorizer.fit_transform(df_reviews["all_reviews"])

#split off business ID
col_id=df_reviews['business_id']
#create dataframe from vectored reviews
df_vector_reviews=pd.DataFrame(vector_reviews.todense())
#concat to single dataframe
df_tfidf_reviews=pd.concat([col_id,df_vector_reviews],axis=1)

display(df_tfidf_reviews)

MemoryError: 

In [ ]:
#join total reviews with aggregated review data
df_join=pd.concat([df, df_tfidf_reviews], axis=1, join='inner')
#drop unnecesary info
df_sklearn_ready=df_join.drop(columns=['business_id', 'text'])
df_sklearn_ready

Y is our list of reviews and X is your table of language data.

In [ ]:
#get arrays for training
y=df_sklearn_ready["stars"].values
x=df_sklearn_ready.drop(["stars"], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Now the data is clean and it's time to feed it to the AI

# Machine learning
## TensorFlow Regression
Since the idea is to make a bunch of models to test, to start here are some functions to repeatedly create and test models

In [ ]:
#function to make model
def make_model(x,y,firstLayer=25,secondLayer=10,epochs=50, optimizer='adam', activation='relu',
               stopEarly=1, monitor='loss', min_delta=0.001, patience=2, verbose=2):
    model = Sequential()

    #Set up layers
    model.add(Dense(25, input_dim=x.shape[1], activation=activation)) # Hidden 1 
    model.add(Dense(10, activation)) # Hidden 2
    model.add(Dense(1)) # Output

    model.compile(loss='mean_squared_error', optimizer=optimizer)

    #set up early stop parameters
    if(stopEarly):
        earlyStop = EarlyStopping(monitor=monitor, min_delta=min_delta, patience=patience, verbose=verbose, mode='auto')  
        model.fit(x, y, verbose=verbose, epochs=epochs, callbacks=[earlyStop])
    else:
        model.fit(x, y, verbose=verbose, epochs=epochs)
        
    return model

## Testing with RSME Score

In [ ]:
#function to test a bunch of models and return the best one
def test_models(models, x, y):
    #used to return best model, but used too much memory
    best_i=-1
    i=0
    best_rmse=5;
    for model in models:
        pred=model.predict(x)
        #compare actual to predicted
        score = np.sqrt(metrics.mean_squared_error(pred,y))
        print(score)
        if(score<best_rmse):
            best_i=i
            best_rmse=score
        i=i+1
    return models[best_i]

Create models with different control variables, several times

## Experimentation
Now we will creatie many models with different starting points, attributes, and algorithms.

First we will test adam vs. sgd

In [ ]:
#slower sg models
sgd_models=[]
for i in range(3):
    sgd_models.append(make_model(x_train,y_train, optimizer='sgd'))

In [ ]:
sgd_final=test_models(sgd_models,x_test,y_test)

In [ ]:
#control
control_models=[]
for i in range(4):
    control_models.append(make_model(x_train,y_train, verbose=0))

In [ ]:
control_final=test_models(control_models,x_test,y_test)

SGD is much more effective, so future tests will use it.

Next we will compare activation algorithms

In [ ]:
#sigmoid models
sigmoid_models=[]
for i in range(3):
    sigmoid_models.append(make_model(x_train,y_train, optimizer='sgd', activation='sigmoid'))

In [ ]:
sigmoid_final=test_models(sigmoid_models,x_test,y_test)

In [ ]:
#tanh models
tanh_models=[]
for i in range(3):
    tanh_models.append(make_model(x_train,y_train, optimizer='sgd', activation='tanh'))

In [ ]:
tanh_final=test_models(tanh_models,x_test,y_test)

Sigmoid is the better activation to use, so we will stick with that.

Next we will play with number of epochs.

In [ ]:
#more epochs
slow_models=[]
for i in range(3):
    slow_models.append(make_model(x_train,y_train, epochs=100, optimizer='sgd', activation='sigmoid'))

In [ ]:
slow_final=test_models(slow_models,x_test,y_test)

In [ ]:
#way more epochs
x_slow_models=[]
for i in range(3):
    x_slow_models.append(make_model(x_train,y_train, epochs=200, optimizer='sgd', activation='sigmoid'))

In [ ]:
x_slow_final=test_models(x_slow_models,x_test,y_test)

Due to early stopping, number of epochs is negligible on results

Next we will play with layer sizes.

In [ ]:
#more neurons
big_models=[]
for i in range(3):
    big_models.append(make_model(x_train,y_train, firstLayer=50, secondLayer=25, optimizer='sgd', activation='sigmoid'))

In [ ]:
big_final=test_models(big_models,x_test,y_test)

In [ ]:
#fewer neurons
small_models=[]
for i in range(3):
    small_models.append(make_model(x_train,y_train, firstLayer=10, secondLayer=5, optimizer='sgd', activation='sigmoid'))

In [ ]:
small_final=test_models(small_models,x_test,y_test)

More neurons helped seems a little better. What if we used *a lot* more?

In [ ]:
#way more neurons
xl_models=[]
for i in range(3):
    xl_models.append(make_model(x_train,y_train, firstLayer=100, secondLayer=50, optimizer='sgd', activation='sigmoid'))

In [ ]:
xl_final=test_models(xl_models,x_test,y_test)

Number of neurons produced a negligable difference in result.

Next let us experiment with number of layers

In [ ]:
parfait_models=[]
for i in range(3):
    model = Sequential()

    #Set up layers
    model.add(Dense(25, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1 
    model.add(Dense(20, 'sigmoid')) # Hidden 2
    model.add(Dense(10, 'sigmoid')) # Hidden 3
    model.add(Dense(1)) # Output

    model.compile(loss='mean_squared_error', optimizer='sgd')

    #set up early stop parameters
    earlyStop = EarlyStopping(monitor='loss', min_delta=0.001, patience=2, verbose=0, mode='auto')  
    model.fit(x_train, y_train, verbose=0, epochs=50, callbacks=[earlyStop])
    parfait_models.append(model)

In [ ]:
parfait_final=test_models(parfait_models,x_test,y_test)

In [ ]:
onion_models=[]
for i in range(3):
    model = Sequential()

    #Set up layers
    model.add(Dense(50, input_dim=x.shape[1], activation='sigmoid')) # Hidden 1 
    model.add(Dense(25, 'sigmoid')) # Hidden 2
    model.add(Dense(15, 'sigmoid')) # Hidden 3
    model.add(Dense(1)) # Output

    model.compile(loss='mean_squared_error', optimizer='sgd')

    #set up early stop parameters
    earlyStop = EarlyStopping(monitor='loss', min_delta=0.001, patience=2, verbose=0, mode='auto')  
    model.fit(x, y, verbose=0, epochs=50, callbacks=[earlyStop])
    onion_models.append(model)

In [ ]:
onion_final=test_models(onion_models,x_test,y_test)

Number of layers seems negiligible with the current settings.

Next let's play with the min delta and pateince variables

In [ ]:
#higher patience value
patient_models=[]
for i in range(3):
    patient_models.append(make_model(x_train,y_train, patience=5, optimizer='sgd', activation='sigmoid'))

In [ ]:
patient_final=test_models(patient_models,x_test,y_test)

In [ ]:
#lower pateince value
impatient_models=[]
for i in range(3):
    impatient_models.append(make_model(x_train,y_train,  patience=1, optimizer='sgd', activation='sigmoid'))

In [ ]:
impatient_final=test_models(impatient_models,x_test,y_test)

In [ ]:
#smaller min delta value
ten_k_models=[]
for i in range(3):
    ten_k_models.append(make_model(x_train,y_train, min_delta=0.0001, optimizer='sgd', activation='sigmoid'))

In [ ]:
ten_k_final=test_models(ten_k_models,x_test,y_test)

In [ ]:
#larger min delta value
hun_models=[]
for i in range(3):
    hun_models.append(make_model(x_train,y_train, min_delta=0.01, patience=5, optimizer='sgd', activation='sigmoid'))

In [ ]:
hun_final=test_models(hun_models,x_test,y_test)

In [ ]:
#smaller min delta value AND more patient
patient_10k_models=[]
for i in range(3):
    patient_10k_models.append(make_model(x_train,y_train, min_delta=0.0001, patience=5, optimizer='sgd', activation='sigmoid'))

In [ ]:
patient_10k_final=test_models(patient_10k_models,x_test,y_test)

Now that we've picked the best configuration, let's run it from multiple starting points to get the best possible model.

In [ ]:
best_models=[]
for i in range(10):
    best_models.append(make_model(x_train,y_train, verbose=0, firstLayer=100, secondLayer=50, 
                                min_delta=0.0001, optimizer='sgd', activation='sigmoid'))
best_model=test_models(best_models,x_test,y_test)

In [ ]:
#function to make model


model = Sequential()

#Set up layers
model.add(Dense(25, input_dim=x.shape[1], activation=activation)) # Hidden 1 
model.add(Dense(10, activation)) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer=optimizer)

  #set up early stop parameters
earlyStop = EarlyStopping(monitor=monitor, min_delta=min_delta, patience=patience, verbose=verbose, mode='auto') 

In [ ]:
model=make_model(x_train,y_train)

In [ ]:
pred=model.predict(x_test)
chart_regression(pred.flatten(),y_test,sort=False)

## Save the best one

In [ ]:
import os
# save entire network to HDF5
best_model.save("network.hdf5")

In [ ]:
from tensorflow.keras.models import load_model

model2 = load_model("network.hdf5")
pred = model2.predict(x_test)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("After load score (RMSE): {}".format(score))

In [ ]:
beep()